In [ ]:
#!jupyter nbconvert --to html Machine-Learning-Challenge-CloudtoStreet.ipynb

In [ ]:
#!kill -9 -1

In [1]:
pip install rasterio

     |████████████████████████████████| 19.3MB 1.2MB/s 


In [4]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=0316f8942304fc3bdc916a75aa3107efecce02acc05265373eea4bc8b5d6281b
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


IndexError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# these are the packages we need for exploration, please install them e.g. in colab you need !pip install rasterio
import os
import glob
import matplotlib.pyplot as plt
import tifffile
import numpy as np

from matplotlib.collections import PatchCollection
from shapely.geometry import Polygon
from rasterio.features import shapes
from descartes import PolygonPatch

In [4]:
import torch
from torchvision import datasets, transforms, models
import cv2
from torch.utils.data import Dataset,random_split
import random
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [5]:
import sys
sys.path.append('/content/drive/My Drive/CloudtoStreet/')
from models.unet_parts import *
from models.unet_model import UNet
from torch import optim

# Setup

We will use our [Sen1Floods11](https://github.com/cloudtostreet/Sen1Floods11) dataset. We will create a local folder `c2s_data/v1.1/data/flood_events/HandLabeled` and download the handlabeled training data (Sentinel-1 512x512 chips, mask labels) there.

In [ ]:
#os.makedirs("/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled", exist_ok=True)

#!pip install gsutil
##!gsutil -m rsync -r gs://sen1floods11/v1.1/data/flood_events/HandLabeled c2s_data/v1.1/data/flood_events/HandLabeled
#!gsutil -m rsync -r gs://sen1floods11/v1.1/data/flood_events/HandLabeled "/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled"

There are some faulty labels we will remove here.

In [ ]:
def remove_faulty(img_folder="/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/S1Hand", 
                  label_folder="/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/LabelHand"):
    for uid in [
        'Ghana_141910',
        'Ghana_866994',
        'Mekong_1111068',
        'Pakistan_35915',
        'Pakistan_94095',
        'Paraguay_34417',
        'Paraguay_581976',
        'Spain_6095801'
    ]:
        
        faulty_img = f"{img_folder}/{uid}_S1Hand.tif"
        if os.path.exists(faulty_img):
            print('removing: ', faulty_img)
            os.remove(faulty_img)

        faulty_label = f"{label_folder}/{uid}_LabelHand.tif"
        if os.path.exists(faulty_label):
            print('removing: ', faulty_label)
            os.remove(faulty_label)

#remove_faulty()

## Helper functions

In [ ]:
def get_patchcollection_from_label(label, label_value=1, alpha=1, color="cyan"):
    """Turns a (H,W) mask into a PatchCollection of polygons"""
    polygons, colors = [], []
    for geom in shapes((label == label_value).astype(np.uint8), mask=(label == label_value).astype(np.uint8),
                       connectivity=4):
        poly = Polygon(geom[0]['coordinates'][0], holes=geom[0]['coordinates'][1:])
        polygons.append(PolygonPatch(poly.buffer(0)))
        colors.append(color)
    return PatchCollection(polygons, facecolor=colors, linewidths=0, alpha=alpha)

def visualize_s1_img(path):
    """Opens a Sentinel 1 image and returns a scaled RFCC false color composite image for visualization."""
    s1_img = tifffile.imread(path)
    if s1_img.shape[0] < 15: s1_img = np.transpose(s1_img, (1, 2, 0))
    if s1_img.shape[-1] == 3: s1_img = s1_img[:, :, :2]
    img = np.zeros((s1_img.shape[0], s1_img.shape[1], 3), dtype=np.float32)
    img[:, :, :2] = s1_img
    img[:, :, 2] = s1_img[:, :, 0] / s1_img[:, :, 1]
    return scale_S1_S2_img(img)

def scale_S1_S2_img(matrix):
    """Returns a scaled (H,W,D) image which is more easily visually inspectable. Image is linearly scaled between
    min and max_value of by channel"""
    w, h, d = matrix.shape
    min_values = np.array([-23, -28, 0.2])
    max_values = np.array([0, -5, 1])

    matrix = np.reshape(matrix, [w * h, d]).astype(np.float64)
    matrix = (matrix - min_values[None, :]) / (max_values[None, :] - min_values[None, :])
    matrix = np.reshape(matrix, [w, h, d])

    matrix = matrix.clip(0, 1)
    return matrix

# Part 1: Practical Deep Learning

## Exploration and Splitting

In the folder "S1Hand" we have 438 512x512 Sentinel-1 images from 11 countries. The folder "LabelHand" contains the water/no data masks for these images. 

Our goal is to have a model that achieves high mean Intersection over Union over all countries. 

__Todo__
- Look at the images with the code that is given. Re-run it to see different random examples from the dataset.
- Explore the images and labels. Summarize your findings with relevant statistics and visualizations.
- Split the data into useful train/validation/test datasets

In [ ]:
# Countries we have data for
print(set([k.split("/")[-1].split("_")[0] for k in glob.glob("/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/S1Hand/*.tif")]))

In [ ]:
S1_imgs = sorted(glob.glob("/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/S1Hand/*.tif"))
labels_GT = sorted(glob.glob("/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/LabelHand/*.tif"))
print(len(S1_imgs), len(labels_GT))

For Visualization we are using a false color composite of the two Sentinel-1 channels and some color stretching. 

The label masks are 1 for water pixels (shown in cyan) and -1 for no data, e.g. clouds on the Sentinel-2 image we labeled the data on (shown in magenta).

In [ ]:
# plot random tiles
# S1 image on the left, and the S1 image with the hand-labels overlain on the right.
show_how_many = 1
color = "cyan"
no_data_color = "magenta"
alpha = 0.5
for k in range(show_how_many):
    rand_int = np.random.randint(len(S1_imgs))
    filename = S1_imgs[rand_int].split("/")[-1].split(".")[0] + ".png"
    f, axarr = plt.subplots(1, 2, figsize=(20, 10))
    print(filename)
    print(tifffile.imread(S1_imgs[rand_int]).min(), tifffile.imread(S1_imgs[rand_int]).max())
    img = visualize_s1_img(S1_imgs[rand_int])
    print(img.min(),img.max())
    axarr[0].imshow(img)
    axarr[0].set_title("S1 image")
    axarr[1].imshow(img)
    axarr[1].set_title("S1 image with Ground Truth")
         
    label = tifffile.imread(labels_GT[rand_int])
    p = get_patchcollection_from_label(label, alpha=alpha, color=color)
    axarr[1].add_collection(p)
    p = get_patchcollection_from_label(label, -1, alpha=alpha, color=no_data_color)
    axarr[1].add_collection(p)
    
    plt.tight_layout()
    plt.show()

NameError: ignored

In [ ]:
from collections import Counter
countries = Counter(([k.split("/")[-1].split("_")[0] for k in glob.glob("/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/S1Hand/*.tif")]))

In [ ]:
countries

In [ ]:
label = tifffile.imread(labels_GT[100])
np.unique(label)

In [ ]:
print(label)

In [ ]:
img =  tifffile.imread(S1_imgs[110])
np.unique(img)

In [ ]:
np.count_nonzero(label == 1)

In [ ]:
np.count_nonzero(label == 0)

In [ ]:
np.count_nonzero(label == -1)

In [ ]:
#zero_count=0
#p1_count=0
#n1_count=0
#for k in glob.glob("/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/LabelHand/*.tif"):
#  array = np.array(tifffile.imread(k))
#  p1_count += np.count_nonzero(array == 1)
#  n1_count += np.count_nonzero(array == -1)
#  zero_count += np.count_nonzero(array == 0)


In [ ]:
#print(p1_count/(p1_count+n1_count+zero_count))
#print(zero_count/(p1_count+n1_count+zero_count))
#print(n1_count/(p1_count+n1_count+zero_count))

In [ ]:
class Sentinel_Loader(Dataset):
	
  def __init__(self, data_path):
    self.data_path = data_path
    self.imgs_path = glob.glob(os.path.join(data_path, 'S1Hand/*.tif'))
#    self.transform = transforms.Compose([transforms.ToTensor()])

  def __getitem__(self, index):
    image_path = self.imgs_path[index]
    label_path = image_path.replace('S1Hand', 'LabelHand')

    image = tifffile.imread(image_path)
    label = tifffile.imread(label_path)
    image = image.reshape(2, image.shape[1], image.shape[2])
    label = label.reshape(label.shape[0], label.shape[1])
    image = (image - np.nanmin(image))/(np.nanmax(image) - np.nanmin(image))
    image[image != image] = 2
#    print(image.min(), image.max())
    label = np.where(label == -1, 2, label)
    return image, label
  
  def __len__(self):
    return len(self.imgs_path)
	
if __name__ == "__main__":
  sentinel_dataset = Sentinel_Loader("/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/")
  print("Dataset_count", len(sentinel_dataset))
  train_loader = torch.utils.data.DataLoader(dataset=sentinel_dataset,
                                               batch_size=1, 
                                               shuffle=True)
  for image, label in train_loader:
    print(image.shape)

Dataset_count 438
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 512])
torch.Size([1, 2, 512, 51

In [ ]:
sentinel_dataset = Sentinel_Loader("/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/")

In [ ]:
#sentinel_dataset = Sentinel_Loader("/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/")
#print("Dataset_count", len(sentinel_dataset))
#train_loader = torch.utils.data.DataLoader(dataset=sentinel_dataset,
#                                               batch_size=4, 
#                                               shuffle=True)
#for image, label in train_loader:
#  print(image.shape)
#  print(label.shape)
#  print(label.type())

In [ ]:
def load_split_train_test(datadir, valid_size = .2, batch_size=4):

    data_transforms = transforms.Compose([transforms.ToTensor()])
        
    S1_imgs = sorted(glob.glob(datadir+"S1Hand/*.tif"))

#    train_data = datasets.ImageFolder(datadir, transform=data_transforms)
#    test_data = datasets.ImageFolder(datadir, transform=data_transforms)
                              
    num_train = len(S1_imgs)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = DataLoader(dataset=sentinel_dataset,
                   sampler=train_sampler, batch_size=batch_size)
    testloader = DataLoader(dataset=sentinel_dataset,
                   sampler=test_sampler, batch_size=batch_size)
    return trainloader, testloader

In [ ]:
trainloader, testloader = load_split_train_test("/content/drive/My Drive/c2s_data/v1.1/data/flood_events/HandLabeled/", .2, 4)

In [ ]:
print(len(trainloader))
print(len(testloader))

88
22


## Training

__Todo__
- Train a segmentation model in a deep learning framework of your choice (preferably PyTorch) using your train/val/test splits and the S1 images and masks. Validate on mean country IoU and save your best models for validation and testing.
- Think about and implement how to treat the "no data" pixel values.

Reminder: If you do not have a GPU at hand, you can use https://www.kaggle.com/kernels, https://colab.research.google.com/ for free GPUs or rent a cheap GPU at https://vast.ai/console/create/.

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
device

device(type='cpu')

In [ ]:
model = UNet(n_channels=2, n_classes=3)
model = model.to(device)

In [7]:
def _fast_hist(label_true, label_pred, n_class):
#    print(label_true.shape)
#    print(label_pred.shape)
    mask = (label_true >= 0) & (label_true < n_class)
    hist = np.bincount(
        n_class * label_true[mask].astype(int) +
        label_pred[mask].astype(int), minlength=n_class ** 2).reshape(n_class, n_class)
    return hist

def label_accuracy_score(label_trues, label_preds, n_class):
    """Returns accuracy score evaluation result.
      - overall accuracy
      - mean accuracy
      - mean IU
      - fwavacc
    """
    hist = np.zeros((n_class, n_class))
    for lt, lp in zip(label_trues, label_preds):
        hist += _fast_hist(lt.flatten(), lp.flatten(), n_class)
    acc = np.diag(hist).sum() / hist.sum()
    acc_cls = np.diag(hist) / hist.sum(axis=1)
    acc_cls = np.nanmean(acc_cls)
    iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
    mean_iu = np.nanmean(iu)
    freq = hist.sum(axis=1) / hist.sum()
    fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
    return acc, acc_cls, mean_iu, fwavacc

In [ ]:
def train_net(model, device, trainloader, epochs=80, lr=0.00001):

  optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
#  criterion = nn.BCEWithLogitsLoss()
  criterion = nn.CrossEntropyLoss()
  best_loss = float('inf')
  for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_acc = 0
    train_mean_iu = 0
    for image, label in trainloader:
      optimizer.zero_grad()
      image = image.to(device=device, dtype=torch.float32)  
      label = label.to(device=device, dtype=torch.long)

      pred = model(image)
      pred = F.log_softmax(pred, dim=1)
      loss = criterion(pred, label)
      train_loss += loss.item()
      label_pred = pred.max(dim=1)[1].data.cpu().numpy()
      label_true = label.data.cpu().numpy()

      for lbt, lbp in zip(label_true, label_pred):
        acc, _, mean_iu, _ = label_accuracy_score(lbt, lbp, 3)
        train_acc += acc
        train_mean_iu += mean_iu

    if loss < best_loss:
      best_loss = loss
      torch.save(model.state_dict(), '/content/drive/My Drive/CV/best_model.pth')
    loss.backward()
    optimizer.step()


#### Evaluation
    model = model.eval()
    eval_loss = 0
    eval_acc = 0
    eval_mean_iu = 0
    for image, label in testloader:
      image = image.to(device=device, dtype=torch.float32)  
      label = label.to(device=device, dtype=torch.long)

      pred = model(image)
      pred = F.log_softmax(pred, dim=1)
      loss = criterion(pred, label)
      eval_loss += loss.item()
      label_pred = pred.max(dim=1)[1].data.cpu().numpy()
      label_true = label.data.cpu().numpy()

      for lbt, lbp in zip(label_true, label_pred):
        acc, _, mean_iu, _ = label_accuracy_score(lbt, lbp, 3)
        eval_acc += acc
        eval_mean_iu += mean_iu

    epoch_str = ('Epoch: {}, Train Loss: {:.5f}, Train Acc: {:.5f}, Train Mean IU: {:.5f}, Valid Loss: {:.5f}, Valid Acc: {:.5f}, Valid Mean IU: {:.5f}'.format(
              epoch, train_loss / len(trainloader), train_acc / len(trainloader), train_mean_iu / len(trainloader),
              eval_loss / len(testloader), eval_acc / len(testloader), eval_mean_iu / len(testloader)
              ))
    print(epoch_str)


In [ ]:
train_net(model, device, trainloader)

IndexError: ignored

## Testing/Visualization/Interpretation

__Todo__
- Test your model on the test data and visualize some predictions.
- What strengths and weaknesses does it have? Give specific examples.
- What would be your next steps and biggest levers to improve performance?

In [ ]:
# plot random tiles
# S1 image on the left, and the S1 image with the hand-labels overlain on the right.
show_how_many = 1
color = "cyan"
no_data_color = "magenta"
alpha = 0.5
for k in range(show_how_many):
    rand_int = np.random.randint(len(S1_imgs))
    filename = S1_imgs[rand_int].split("/")[-1].split(".")[0] + ".png"
    f, axarr = plt.subplots(1, 3, figsize=(20, 10))
    print(filename)
 #   print(tifffile.imread(S1_imgs[rand_int]).min(), tifffile.imread(S1_imgs[rand_int]).max())
    img = visualize_s1_img(S1_imgs[rand_int])
    axarr[0].imshow(img)
    axarr[0].set_title("S1 image")
    axarr[1].imshow(img)
    axarr[1].set_title("S1 image with Ground Truth")
    axarr[2].imshow(img)
    axarr[2].set_title("S1 image with Prediction")

    label = tifffile.imread(labels_GT[rand_int])
    p = get_patchcollection_from_label(label, alpha=alpha, color=color)
    axarr[1].add_collection(p)
    p = get_patchcollection_from_label(label, -1, alpha=alpha, color=no_data_color)
    axarr[1].add_collection(p)
    
 #   print(S1_imgs[rand_int])
    plot_sampler = SubsetRandomSampler(rand_int)
    plot_loader = DataLoader(dataset=sentinel_dataset, sampler=plot_sampler, batch_size=4)    
    
    for image, label in trainloader:
      image = image.to(device=device, dtype=torch.float32)  
      pred = model(image)
      pred = F.log_softmax(pred, dim=1)
      label_pred = pred.max(dim=1)[1].data.cpu().numpy()
    p = get_patchcollection_from_label(label_pred, alpha=alpha, color=color)
    axarr[2].add_collection(p)
    p = get_patchcollection_from_label(label_pred, -1, alpha=alpha, color=no_data_color)
    axarr[2].add_collection(p)

    plt.tight_layout()
    plt.show()

# Part 2: Implementing a custom model for non-standard tasks

Now, we want to design and test our own custom model that uses a standard U-Net architecture, but has custom inputs and outputs. As a starting point you can use any open-source implementation of U-Net like from [here](https://github.com/milesial/Pytorch-UNet).

__Todo: Change a standard U-Net model to work with the following non-standard inputs and outputs in a deep learning framework of your choice (preferably PyTorch).__

- __inputs__: a 10m resolution SAR image (2 channels), a 20m resolution Digital Elevation Model (1 channel) and a sequence of 8 passive microwave readings (1 channel per reading) at a 160m resolution
- __outputs__: a 10m resolution binary water segmentation mask and a 10m resolution binary cloud segmentation mask of the same height/width as the input SAR image
- test your model with input tensors with random data in it to see if intermediate and output shapes are correct

In [ ]:
from torch.autograd import Variable

input_sar = Variable(torch.rand(2, 512, 512))
input_dem = Variable(torch.rand(1, 256, 256))
input_microwave = Variable(torch.rand(8, 32, 32))

In [ ]:
model = UNet(n_channels=2, n_classes=3)
model = model.to(device)

In [ ]:
import torch.nn.functional as F
from models.unet_parts import *
from torch import nn

class C1_UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(C1_UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, input1, input2, input3):

        m = nn.AdaptiveMaxPool2d((input1.shape[1],input1.shape[2] ))
        print(input1.shape)
        output2 = m(input2)
        output3 = m(input3)
        print(output2.shape)
        print(output3.shape)
        x = torch.cat((input1,output2,output3),dim=0)
        x = x.reshape(1, self.n_channels, x.shape[1], x.shape[2])

        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits[:,0,:,:], logits[:,1,:,:]

if __name__ == '__main__':
    net = C1_UNet(n_channels=11, n_classes=2)
#    print(net)

In [ ]:
m1 = C1_UNet(n_channels=11, n_classes=2)
m1 = m1.to(device)
input1 = input_sar.to(device=device, dtype=torch.float32)
input2 = input_dem.to(device=device, dtype=torch.float32)
input3 = input_microwave.to(device=device, dtype=torch.float32)
output1, output2 = m1(input1, input2, input3)
print(output1.shape, output2.shape)

In [ ]:
# SPP layers implementation